# Zero Dependencies

## Setup

In [1]:
import json
from collections import Counter

In [2]:
dependency_chain_path = '../data/dependency_chain10.json'
most_depended_upon_path = '../data/most_depended_upon10.json'
dependency_chain_path100 = '../data/dependency_chain100.json'
most_depended_upon_path100 = '../data/most_depended_upon100.json'
dependency_chain_path500 = '../data/dependency_chain500.json'
most_depended_upon_path500 = '../data/most_depended_upon500.json'
registry_entries = '../data/registry_entries.json'

In [3]:
registry_map = {}

for p in json.load(open(registry_entries)):
    p_name = p['name']
    p_data = {}
    p_data['versions'] = p['versions']
    registry_map[p_name] = p_data

## Most depended upon libraries from Libraries.io

In [4]:
#the libraries.io info for each package
most_depended_upon = json.load(open(most_depended_upon_path))
most_depended_upon100 = json.load(open(most_depended_upon_path100))
most_depended_upon500 = json.load(open(most_depended_upon_path500))

print(len(most_depended_upon))
print(len(most_depended_upon100))
print(len(most_depended_upon500))

10
100
500


## Chain statistics

In [27]:
dependency_chain = json.load(open(dependency_chain_path))

roots_t = []
root_zeros_t = []
zeros_t = []
parents_t = []
nodes_t = []
circulars_t = []
def walk_chain(chain):
    for d in chain:
        parents_t.append(d)
        if isinstance(d, dict):
            node = list(d.keys())[0]
            d = d[node]
            nodes_t.append(node)
            if len(parents_t) == 1:
                roots_t.append(node)
            walk_chain(d)
        elif isinstance(d, str):
            node = d
            zeros_t.append(node)
            if len(parents_t) == 1:
                root_zeros_t.append(node)
                roots_t.append(node)
            nodes_t.append(node)
            if node.endswith('...'):
                circulars_t.append(node)
        else:
            walk_chain(d)
        parents_t.pop()


walk_chain(dependency_chain)

In [28]:
roots = roots_t
root_zeros = root_zeros_t
zeros = zeros_t
parents = parents_t
nodes = nodes_t
circulars = circulars_t
print('Root nodes:', len(roots))
print('Size of chain:', len(nodes))

average_dependency_count = (len(nodes) - len(roots)) / len(roots)
print("Average dependency chain for root nodes:", average_dependency_count)

unique_nodes = Counter(nodes)
print('Unique packages on chain:', len(unique_nodes))

print('Most frequent on chain:', unique_nodes.most_common(1)[0])

print('Circular dependencies?:', len(circulars))

print(root_zeros)

# print(json.dumps(nodes, indent=1))

Root nodes: 10
Size of chain: 504
Average dependency chain for root nodes: 49.4
Unique packages on chain: 216
Most frequent on chain: ('@webassemblyjs/helper-wasm-bytecode', 19)
Circular dependencies?: 0
['typescript', 'eslint-plugin-react-hooks', 'moment', 'prettier', '@types/lodash']


### Top 100

For a larger sample, let's look at 100 as well.

In [21]:
dependency_chain100 = json.load(open(dependency_chain_path100))
roots_t = []
root_zeros_t = []
zeros_t = []
parents_t = []
nodes_t = []
circulars_t = []
walk_chain(dependency_chain100)

In [22]:
roots100 = roots_t
root_zeros100 = root_zeros_t
zeros100 = zeros_t
parents100 = parents_t
nodes100 = nodes_t
circulars100 = circulars_t

print('Root nodes:', len(roots100))
print('Size of chain:', len(nodes100))

average_dependency_count = (len(nodes100) - len(roots100)) / len(roots100)
print("Average dependency chain for root nodes:", average_dependency_count)

unique_nodes100 = Counter(nodes100)
print('Unique packages on chain:', len(unique_nodes100))

print('Most frequent on chain:', unique_nodes100.most_common(1)[0])

print('Circular dependencies?:', len(circulars100))

# print(roots)
# print([p['name'] for p in most_depended_upon100])

Root nodes: 100
Size of chain: 333800
Average dependency chain for root nodes: 3337.0
Unique packages on chain: 1479
Most frequent on chain: ('function-bind', 42526)
Circular dependencies?: 283


## For the most depended upon libraries, how many have zero dependencies? What are the packages at the end of the dependency chain (zero dependencies)?

1. Get the most popular packages from `libraries.io` (rank: dependant count).

2. Get the dependency chain using the NPM registry for the latest version of each package. We ignore dev dependencies (only used in development) and peer dependencies (which may not be directly relied upon). Libraries.io only gives a single combined number, so any packages that with test suites for example will have dependencies.

In [23]:
unique_zeros = list(dict.fromkeys(zeros))

print("Zero dependencies in chain:", len(zeros), '/', len(nodes))
print("Unique zero dependencies in chain:", len(unique_zeros), '/', len(unique_nodes))
print("Zero dependency roots:", len(root_zeros), "/", len(dependency_chain), '(', len(root_zeros)/len(dependency_chain), ')')

print("\nPopular zeros:", json.dumps(root_zeros, indent=1))
# json.dump(root_zeros, open('../data/zero10.json', 'w'), indent=1)

#print("\nFinal nodes:", json.dumps(unique_zeros, indent=1))
# json.dump(unique_zeros, open('../data/zeroOnChain10.json', 'w'), indent=1)


Zero dependencies in chain: 304 / 504
Unique zero dependencies in chain: 115 / 216
Zero dependency roots: 5 / 10 ( 0.5 )

Popular zeros: [
 "typescript",
 "eslint-plugin-react-hooks",
 "moment",
 "prettier",
 "@types/lodash"
]


### Top 100

In [25]:
unique_zeros100 = list(dict.fromkeys(zeros100))

print("Zero dependencies in chain:", len(zeros100), '/', len(nodes100))
print("Unique zero dependencies in chain:", len(unique_zeros100), '/', len(unique_nodes100))
print("Zero dependency roots:", len(root_zeros100), "/", len(dependency_chain100), '(', len(root_zeros)/len(dependency_chain100), ')')

print("\nPopular zeros:", json.dumps(root_zeros100, indent=1))
json.dump(root_zeros100, open('../data/zero100.json', 'w'), indent=1)

#print("\nFinal nodes:", json.dumps(unique_zeros, indent=1))
json.dump(unique_zeros100, open('../data/zeroOnChain100.json', 'w'), indent=1)

Zero dependencies in chain: 196365 / 333800
Unique zero dependencies in chain: 599 / 1479
Zero dependency roots: 23 / 100 ( 0.05 )

Popular zeros: [
 "typescript",
 "eslint-plugin-react-hooks",
 "moment",
 "prettier",
 "@types/lodash",
 "@types/chai",
 "@types/mocha",
 "async",
 "commander",
 "core-js",
 "uuid",
 "react-is",
 "bootstrap",
 "redux",
 "date-fns",
 "chalk",
 "bluebird",
 "classnames",
 "eslint-config-prettier",
 "style-loader",
 "@angular/language-service",
 "jquery",
 "lodash.debounce"
]


## How many used to have dependencies?

### Root nodes?

In [109]:
reduced = {}

for p in root_zeros:
    has_dependencies = False
    versions = registry_map[p]['versions']
    v_data = {}
    for v in versions:
        dependencies = {}
        if 'dependencies' in versions[v]:
            dependencies = versions[v]['dependencies']
            if dependencies :
                has_dependencies = True
                dependencies = list(dependencies.keys())
                v_data[v] = dependencies
    if has_dependencies:
        reduced[p] = v_data

print('packages that reduced dependencies:', json.dumps(list(reduced.keys()), indent=1))

# print(json.dumps(reduced['prettier'], indent=1))

packages that reduced dependencies: [
 "typescript",
 "prettier"
]


### End of the line?

In [146]:
reduced_ends = {}

for p in zeros:
    has_dependencies = False
    versions = registry_map[p]['versions']
    v_data = {}
    for v in versions:
        dependencies = {}
        if 'dependencies' in versions[v]:
            dependencies = versions[v]['dependencies']
            if dependencies :
                has_dependencies = True
                dependencies = list(dependencies.keys())
                v_data[v] = dependencies
    if has_dependencies:
        reduced_ends[p] = v_data

print('Packages that reduced dependencies', len(reduced_ends))
# print('List:', json.dumps(list(reduced_ends.keys()), indent=1))

# json.dump(list(reduced_ends.keys()), open('../data/reducedOnChain10.json', 'w'), indent=1)

print(json.dumps(reduced_ends['argparse'], indent=1))

Packages that reduced dependencies 34
{
 "0.1.0": [
  "underscore",
  "underscore.string"
 ],
 "0.1.1": [
  "underscore",
  "underscore.string"
 ],
 "0.1.2": [
  "underscore",
  "underscore.string"
 ],
 "0.1.3": [
  "underscore",
  "underscore.string"
 ],
 "0.1.4": [
  "underscore",
  "underscore.string"
 ],
 "0.1.5": [
  "underscore",
  "underscore.string"
 ],
 "0.1.6": [
  "underscore",
  "underscore.string"
 ],
 "0.1.7": [
  "underscore",
  "underscore.string"
 ],
 "0.1.8": [
  "underscore",
  "underscore.string"
 ],
 "0.1.9": [
  "underscore",
  "underscore.string"
 ],
 "0.1.10": [
  "underscore",
  "underscore.string"
 ],
 "0.1.11": [
  "underscore",
  "underscore.string"
 ],
 "0.1.12": [
  "underscore",
  "underscore.string"
 ],
 "0.1.13": [
  "underscore",
  "underscore.string"
 ],
 "0.1.14": [
  "underscore",
  "underscore.string"
 ],
 "0.1.15": [
  "underscore",
  "underscore.string"
 ],
 "0.1.16": [
  "underscore",
  "underscore.string"
 ],
 "1.0.0": [
  "lodash",
  "sprintf-